In [11]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from nltk.sentiment import vader
import numpy as np

In [55]:
import sys
from libs.writing_utils import get_locations

sys.path.append(os.path.dirname(get_locations()[1]))

from common_modules.common_utils import trends_ta

ModuleNotFoundError: No module named 'libs'

In [3]:
#nltk.download('vader_lexicon')
#nltk.download('stopwords')

In [4]:
df = pd.read_csv('../twitterscraper/tests/data/tweet/bitcoin/historic_scrape/raw/2015-02-01_2015-03-01.csv')

In [5]:
def cleanData(data):
    pattern = [ 'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+',  # URLs'
                '([^a-zA-Z0-9 ]+?)', #anything else except text
                ]

    sub_pattern = re.compile('|'.join(pattern))
    common_w = set(stopwords.words("english"))
    
    if isinstance(data, pd.Series):
        
        data = data.str.lower()
        
        for idx, row in enumerate(data):
            splitRow = row.split(' ')
            splitRow = [w for w in splitRow if not w in common_w] 

            data.iloc[idx] = " ".join(splitRow)
            
        replaced = data.str.replace(sub_pattern, '').str.strip()
    else:

        data = data.lower()
        splitted = data.split(' ')

        newSplit = [w for w in splitted if not w in common_w] 
        word = " ".join(newSplit)
        replaced = re.sub(sub_pattern, '', word).strip()
        
    return replaced

In [6]:
df['Tweet'] = cleanData(df['Tweet'])

In [7]:
# df['Tweet'].apply(lambda x: vader.SentimentIntensityAnalyzer().polarity_scores(x)['compound'])

In [8]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')

In [9]:
df = df.set_index('Time')

In [44]:
def f(x):
    y = pd.Series()
    
    x = x.assign(f = x['Likes'] + x['Replies'] + x['Retweets']).sort_values('f', ascending=False).drop('f', axis=1)
    x['sentiment'] = x['Tweet'].apply(lambda x: vader.SentimentIntensityAnalyzer().polarity_scores(x)['compound'])
    
    Nbullish = sum(x['sentiment'] > 0) #should use something else later
    Nbearish = sum(x['sentiment'] < 0)
    
    y['mean_vader_all'] = x['sentiment'].mean()
    
    #doi.org/10.1016/j.eswa.2016.12.036
    y['n_bullish_all'] = Nbullish
    y['n_bearish_all'] = Nbearish
    
    try:
        y['bullish_ratio_all'] = Nbullish / (Nbullish + Nbearish)
    except:
        y['bullish_ratio_all'] = np.nan
    
    try:
        y['bearish_ratio_all'] = Nbearish / (Nbullish + Nbearish)
    except:
        y['bearish_ratio_all'] = np.nan
        
    y['bullish_index_all'] = np.log((Nbullish + 1)/(Nbearish + 1))
    
    try:
        y['agreement_all'] = 1 - np.sqrt(1-(((Nbullish - Nbearish)/(Nbullish + Nbearish))**2) )
    except:
        y['agreement_all'] = np.nan
        
    try:
        y['spread_all'] = (Nbullish - Nbearish)/(Nbullish + Nbearish)
    except:
        y['spread_all'] = np.nan
    
    xTopAll = x.iloc[:int(x.shape[0] * .15)]
    
    y['mean_vader_top'] = xTopAll['sentiment'].mean()
    y['mean_likes_top'] = xTopAll['Likes'].mean()
    y['mean_replies_top'] = xTopAll['Replies'].mean()
    y['mean_retweets_top'] = xTopAll['Retweets'].mean()
    
    return y

In [45]:
newDf = df.groupby(pd.Grouper(freq='H')).apply(f)

In [47]:
newDf['variation_all'] = newDf['n_bullish_all'].diff()
newDf = newDf.drop(['n_bullish_all', 'n_bearish_all'], axis=1)
newDf['mean_vader_change_top'] = newDf['mean_vader_top'].diff()
#add botorNot too
newDf = newDf.replace(np.nan, 0)

In [ ]:
#from data_adder import trends_ta function and then apply ta to mean_vader_top and mean_vader_all
newDf = trends_ta(newDf, 'mean_vader_top')
newDf = trends_ta(newDf, 'mean_vader_all')